In [21]:
import os
import pandas as pd
import numpy as np
from utils.mapping import mapDataframeToGridClosest
from utils.global_co2 import getGlobalCo2ForYear

In [22]:
# Specify these two varibales
data_path = "../../../../../../Volumes/T7Shield/test"
file_prefix = "ORCA025.L46.LIM2vp.CFCSF6.MOPS.JRA.LP04-KLP002.hind_"
datset_id = "exp1"

In [23]:
output_path = "../../data/" + datset_id

features = ['SST', 'SAL', 'ice_frac', 'mixed_layer_depth', 'heat_flux_down', 'water_flux_up', 'stress_X', 'stress_Y', 'currents_X', 'currents_Y',
            'month','tmask','year','nav_lat', 'nav_lon','global_co2']
targets = ['fco2','fco2_pre','co2flux','co2flux_pre']

def regrid_and_store(year, index):
    input_file = os.path.join(data_path, f"{file_prefix}{year}_df.pkl")
    df = pd.read_pickle(input_file)

    df["year"] = year
    df["month"] = df["time_counter"].apply(lambda x: x.month)
    df['global_co2'] = getGlobalCo2ForYear(year)
    df = df[features+targets]

    if df.isna().any().any():
        print("contains NaN values")
        print(df.isna().sum())

    os.makedirs(output_path, exist_ok=True)

    for month in range(1, 13):
        month_data = df.loc[(df['month'] == month)]

        features_array = mapDataframeToGridClosest(month_data, features)
        target_array = mapDataframeToGridClosest(month_data, targets)

        np.save(os.path.join(output_path, f"{index+month}_features.npy"), features_array)
        np.save(os.path.join(output_path, f"{index+month}_targets.npy"), target_array)

In [ ]:
i = 0

for year in range(1958, 1959):
    print(year)
    regrid_and_store(year, i)
    i = i+12

1958
